# Counting the number of imgs and solar panels

In [1]:
import numpy as np
import os
import pandas as pd
from skimage import io


In [14]:
part1 = '/scratch/sr365/Packaging_catalyst_raw_data/data_raw/part1'
part2 = '/scratch/sr365/Packaging_catalyst_raw_data/data_raw/part2'

def get_height(filename):
    """
    From the file name get the height
    """
    name = filename.split('.')[0].split('m')[0].split('_')[-1]
    height = float(name)/10
    height = int(round(height))
    return height


def calculate_resolution(height):
    # In cm
    return height*6.17/4000/4.3*100*10

from numpy.core.defchararray import array, mod
from numpy.core.numeric import _moveaxis_dispatcher
from skimage import io
import os
import numpy as np
import pandas as pd
import imagesize
from tqdm import tqdm
from mrs_utils.eval_utils import *
from skimage.transform import resize
from shutil import copyfile
from glob import glob 

def count_panels(file):
    obj_scorer = ObjectScorer(min_region=5, min_th=0.5, dilation_size=1, link_r=0, eps=0)
    def get_num_solar_panels(lbl_name, obj_scorer):
        lbl = io.imread(lbl_name)
        if len(np.shape(lbl)) == 3:
            lbl = lbl[:, :, 0]
        reg_props = obj_scorer.get_object_groups(lbl)
        del lbl
        #print('This image has {} solar panels'.format(len(reg_props)))
        return [ a.area for a in reg_props]
    return len(get_num_solar_panels(file, obj_scorer))

hm = {}
# for part in [part1, part2]:
# for part in ['/scratch/sr365/Packaging_catalyst_raw_data/data_raw/part2/moving_imgs/labelled']:
    # for folder in os.listdir(part):
        # cur_folder = os.path.join(part, folder)
cur_folder = '/scratch/sr365/Packaging_catalyst_raw_data/data_raw/part2/moving_imgs/labelled/labels'
# if not os.path.isdir(cur_folder):
#     continue
for file in os.listdir(cur_folder):
    # Get the current file
    cur_file = os.path.join(cur_folder, file)
    # From the file name, get the height
    if '.png' in file:
    # if '.JPG' in file or '.jpg' in file:
    # if '.MP4' in file or '.mp4' in file:
        height = get_height(file)
        if height < 5:
            continue
        PV_count = count_panels(cur_file)
        # PV_count = 1
        if height in hm:
            hm[height] += PV_count
        else:
            hm[height] = PV_count

print(hm)
print(sum(hm.values()))
    

{11: 105, 7: 62, 5: 68, 9: 74, 10: 67, 12: 65, 8: 81, 6: 93}
615


In [15]:
for key in sorted(hm.keys()):
    #print(key, calculate_resolution(key))
    print(key, hm[key])

5 68
6 93
7 62
8 81
9 74
10 67
11 105
12 65


In [ ]:

    ################################################
    # Counting individual villages solar panel num #
    ################################################
    # Read in the ground truth labels
    # for folder in os.listdir(mother_dir):
    #     cur_folder = os.path.join(mother_dir, folder)
    #     # Go into all subfolders
    #     if not os.path.isdir(cur_folder):
    #         continue
    #     for phase_folder in os.listdir(cur_folder):
    #         cur_phase = os.path.join(cur_folder, phase_folder)
    #         if not os.path.isdir(cur_phase):
    #             continue
    #         PV_count = 0
    #         for files in os.listdir(cur_phase):
    #             if not files.endswith('.csv'):
    #                 continue
    #             data = pd.read_csv(os.path.join(cur_phase, files))
    #             data = data['Object'].values
    #             if len(data) > 0:
    #                 PV_count += data[-1]
    #         #     if not files.endswith('.png'):
    #         #         continue
    #         #     cur_file = os.path.join(cur_phase, files)
    #         #     # This is a lbl file
    #         #     num_panels = get_num_solar_panels(cur_file, obj_scorer)
    #         #     PV_count += num_panels
    #         print('Folder {} has {} solar panels'.format(cur_phase, PV_count))

    ##########################################################
    # Counting the whole annotation folder in aggregate form #
    ##########################################################
    size_list = []
    size_warn_list = []
    for file in tqdm(os.listdir(mother_dir)):
        # Skip if not 
        if not file.endswith('.png'):
            continue
        current_panel_list = get_num_solar_panels(os.path.join(mother_dir, file), obj_scorer)
        size_list.extend(current_panel_list)
        # Check for oversized solar panels and make sure they are correct
        if len(current_panel_list) > 0 and np.sum(np.array(current_panel_list) > size_warning) > 0:
            size_warn_list.append(file)
            print('Check!!! There is panel larger than {} pixels in : {}'.format(size_warning, file))
        print('finished counting {}, currently there are {} solar panels'.format(file, len(size_list)))
    print(size_list)
    np.savetxt('/home/sr365/Gaia/Rwanda_RTI/test_patches_panel_size_count.txt', size_list)

    print('printing the oversized solar panel files')
    for file_warn in size_warn_list:
        print(file_warn)